In [1]:

from collections import defaultdict
import re

def analyze_endings(filename, n=2, min_count=5):
    # Словари для статистики
    ending_total = defaultdict(int)
    ending_adj = defaultdict(int)
    
    # Регулярное выражение для разбора строк
    pattern = re.compile(r'^(\S+)\t([A-Z]+),')
    
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or line[0].isdigit():
                continue
                
            match = pattern.match(line)
            if match:
                word = match.group(1).lower()
                pos = match.group(2)
                
                if len(word) < n:
                    continue
                    
                ending = word[-n:]
                ending_total[ending] += 1
                
                # Считаем прилагательные и причастия как определения
                if pos in ['ADJF', 'ADJS', 'PRTF', 'PRTS']:
                    ending_adj[ending] += 1
    
    # Вычисляем вероятности
    results = []
    for ending, total in ending_total.items():
        if total >= min_count:
            adj_count = ending_adj.get(ending, 0)
            probability = adj_count / total
            results.append((ending, probability, total, adj_count))
    
    # Сортируем по убыванию вероятности
    results.sort(key=lambda x: x[1], reverse=True)
    
    return results


    
    

In [3]:
filename = "dict.opcorpora.txt"  # Укажите путь к файлу
n = 3  # Длина окончания для анализа
min_count = 1  # Минимальное количество встречаемости окончания

stats = analyze_endings(filename, n, min_count)

print(f"Статистика для окончаний длиной {n} букв (min встречаемость: {min_count}):")
print("-" * 50)
for ending, prob, total, adj_count in stats:
    print(f"'{ending}': {prob:.1%} ({adj_count}/{total})")

Статистика для окончаний длиной 3 букв (min встречаемость: 1):
--------------------------------------------------
'мко': 100.0% (3/3)
'емо': 100.0% (6575/6575)
'уен': 100.0% (4/4)
'фно': 100.0% (10/10)
'нён': 100.0% (252/252)
'егл': 100.0% (1/1)
'зый': 100.0% (40/40)
'бый': 100.0% (34/34)
'бые': 100.0% (46/46)
'згл': 100.0% (2/2)
'усо': 100.0% (7/7)
'ысо': 100.0% (2/2)
'аен': 100.0% (19/19)
'ёро': 100.0% (5/5)
'щен': 100.0% (131/131)
'щна': 100.0% (25/25)
'щно': 100.0% (25/25)
'щны': 100.0% (25/25)
'тыж': 100.0% (1/1)
'пно': 100.0% (47/47)
'тящ': 100.0% (2/2)
'рзо': 100.0% (1/1)
'щов': 100.0% (3/3)
'зён': 100.0% (19/19)
'сущ': 100.0% (1/1)
'зуч': 100.0% (3/3)
'яво': 100.0% (16/16)
'шён': 100.0% (107/107)
'всё': 100.0% (2/2)
'сех': 100.0% (3/3)
'хое': 100.0% (14/14)
'етх': 100.0% (1/1)
'оён': 100.0% (33/33)
'щён': 100.0% (155/155)
'осл': 100.0% (5/5)
'ято': 100.0% (71/71)
'нюч': 100.0% (2/2)
'хож': 100.0% (7/7)
'еий': 100.0% (10/10)
'еюю': 100.0% (5/5)
'еие': 100.0% (10/10)
'еих': 100.0

In [17]:
from collections import defaultdict
import re

def is_definition(tags):
    """
    Проверяет, является ли слово определением по заданным правилам
    """
    tag_list = tags.split(',')
    
    # Прямые указатели на определение
    direct_indicators = {
        'ADJF',  # имя прилагательное (полное)
        'ADJS',  # имя прилагательное (краткое)
        'PRTF',  # причастие (полное)
        'PRTS',  # причастие (краткое)
        'Anum',  # порядковое числительное
        'Apro',  # местоименное
        'Poss'   # притяжательное
    }
    
    # Косвенные указатели
    indirect_indicators = {
        'Qual',  # качественное
        'Supr',  # превосходная степень
        'COMP',  # компаратив
        'Cmp2'   # сравнительная степень на по-
    }
    
    # Согласовательные категории
    agreement_categories = {
        # Падеж
        'nomn', 'gent', 'datv', 'accs', 'ablt', 'loct', 'voct',
        'gen1', 'gen2', 'acc2', 'loc1', 'loc2',
        # Число
        'sing', 'plur', 'Sgtm', 'Pltm',
        # Род
        'masc', 'femn', 'neut', 'ms-f'
    }
    
    # Проверка прямых указателей
    has_direct = any(tag in direct_indicators for tag in tag_list)
    
    # Проверка косвенных указателей (только если есть хотя бы один прямой)
    has_indirect = any(tag in indirect_indicators for tag in tag_list)
    
    # Проверка согласовательных категорий
    has_agreement = any(tag in agreement_categories for tag in tag_list)
    
    return has_direct # and has_agreement) or (has_direct and has_indirect)

def analyze_definition_endings(filename, n=2, min_count=3):
    """
    Анализирует окончания слов и вычисляет вероятность того, что слово является определением
    """
    # Словари для статистики
    ending_total = defaultdict(int)
    ending_definition = defaultdict(int)
    
    # Регулярное выражение для разбора строк
    pattern = re.compile(r'^(\S+)\t(.+)$')
    
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            # Пропускаем пустые строки и строки с номерами парадигм
            if not line or line[0].isdigit():
                continue
                
            match = pattern.match(line)
            if match:
                word = match.group(1).lower()
                tags = match.group(2)
                
                if len(word) < n:
                    continue
                    
                ending = word[-n:]
                ending_total[ending] += 1
                
                # Проверяем, является ли слово определением
                if is_definition(tags):
                    ending_definition[ending] += 1
    
    # Вычисляем вероятности
    results = []
    for ending, total in ending_total.items():
        if total >= min_count:
            definition_count = ending_definition.get(ending, 0)
            probability = definition_count / total if total > 0 else 0
            results.append((ending, probability, total, definition_count))
    
    # Сортируем по убыванию вероятности
    results.sort(key=lambda x: x[1], reverse=True)
    
    return results

def print_detailed_analysis(filename, n=2, top_k=20):
    """
    Выводит детальный анализ для лучших окончаний
    """
    stats = analyze_definition_endings(filename, n, min_count=1)
    
    print(f"Топ-{top_k} окончаний длиной {n} букв с наибольшей вероятностью быть определением:")
    print("=" * 70)
    print(f"{'Окончание':<10} {'Вероятность':<12} {'Определения':<12} {'Всего':<8} {'Примеры'}")
    print("-" * 70)
    
    # Собираем примеры слов для каждого окончания
    ending_examples = defaultdict(list)
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or line[0].isdigit():
                continue
                
            parts = line.split('\t')
            if len(parts) >= 2:
                word = parts[0].lower()
                if len(word) >= n:
                    ending = word[-n:]
                    if len(ending_examples[ending]) < 3:  # максимум 3 примера
                        ending_examples[ending].append(parts[0])
    
    for i, (ending, prob, total, def_count) in enumerate(stats[:top_k]):
        examples = ", ".join(ending_examples.get(ending, [])[:2])
        print(f"'{ending}':{prob:>8.1%}{def_count:>12}{total:>10}    {examples}")
    #ending, prob, total, def_count
    # Дополнительная статистика
    total_definitions = sum(def_count for _, _, _, def_count in stats)
    total_words = sum(total for _, _, total, _ in stats)
    overall_prob = total_definitions / total_words if total_words > 0 else 0
    
    print("\n" + "=" * 70)
    print(f"Общая статистика:")
    print(f"Всего слов проанализировано: {total_words}")
    print(f"Всего определений найдено: {total_definitions}")
    print(f"Общая вероятность: {overall_prob:.1%}")
    
    # Анализ эффективности при разных порогах точности
    print("\n" + "=" * 70)
    print("АНАЛИЗ ЭФФЕКТИВНОСТИ ПРИ РАЗНЫХ ПОРОГАХ ТОЧНОСТИ")
    print("=" * 70)
    
    # Пороги точности для анализа (от 50% до 95%)
    thresholds = [0.05, 0.1, 0.2, 0.35, 0.4,0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
    
    print(f"{'Порог':<8} {'Окончаний':<12} {'Покрытие':<12} {'Точность':<12} {'Эффективность':<15}")
    print("-" * 70)
    
    for threshold in thresholds:
        # Окончания с вероятностью выше порога
        high_prob_endings = [(ending, prob, total, def_count) 
                           for ending, prob, total, def_count in stats 
                           if prob >= threshold]
         
        if threshold == 0.35:
            stats_by_freq = sorted(high_prob_endings, key=lambda x: x[3], reverse=True)
            print([x[0] for x in stats_by_freq])
        
        if not high_prob_endings:
            continue
            
        # Считаем статистику
        num_endings = len(high_prob_endings)
        total_covered_words = sum(total for _, _, total, _ in high_prob_endings)
        total_covered_defs = sum(def_count for _, _, _, def_count in high_prob_endings)
        
        # Покрытие - доля всех определений, которые покрываются этими окончаниями
        coverage = total_covered_defs / total_definitions if total_definitions > 0 else 0
        
        # Точность - средняя вероятность для этих окончаний (взвешенная по количеству слов)
        weighted_accuracy = sum(prob * total for _, prob, total, _ in high_prob_endings) / total_covered_words
        
        # Эффективность - комбинированная метрика (точность * покрытие)
        efficiency = weighted_accuracy * coverage
        
        print(f"{threshold:<8.0%} {num_endings:<12} {coverage:>11.1%} {weighted_accuracy:>11.1%} {efficiency:>14.1%}")
    
    # Находим оптимальный порог (максимальная эффективность)
    best_threshold = 0
    best_efficiency = 0
    best_stats = None
    
    # Проверяем больше порогов для поиска оптимального
    test_thresholds = [i/100 for i in range(10, 96, 5)]
    for threshold in test_thresholds:
        high_prob_endings = [(ending, prob, total, def_count) 
                           for ending, prob, total, def_count in stats 
                           if prob >= threshold]
        
        if not high_prob_endings:
            continue
            
        total_covered_words = sum(total for _, _, total, _ in high_prob_endings)
        total_covered_defs = sum(def_count for _, _, _, def_count in high_prob_endings)
        
        coverage = total_covered_defs / total_definitions if total_definitions > 0 else 0
        weighted_accuracy = sum(prob * total for _, prob, total, _ in high_prob_endings) / total_covered_words
        efficiency = weighted_accuracy * coverage
        
        if efficiency > best_efficiency:
            best_efficiency = efficiency
            best_threshold = threshold
            best_stats = (len(high_prob_endings), coverage, weighted_accuracy)
    
    if best_stats:
        num_endings, coverage, accuracy = best_stats
        print("\n" + "=" * 70)
        print(f"ОПТИМАЛЬНЫЙ ПОРОГ: {best_threshold:.0%}")
        print(f"Количество окончаний: {num_endings}")
        print(f"Покрытие определений: {coverage:.1%}")
        print(f"Средняя точность: {accuracy:.1%}")
        print(f"Общая эффективность: {best_efficiency:.1%}")
        
        # Показываем лучшие окончания при оптимальном пороге
        print(f"\nЛучшие окончания при пороге {best_threshold:.0%}:")
        high_prob_endings = [(ending, prob, total, def_count) 
                           for ending, prob, total, def_count in stats 
                           if prob >= best_threshold]
        
        for i, (ending, prob, total, def_count) in enumerate(high_prob_endings[:10]):
            examples = ", ".join(ending_examples.get(ending, [])[:2])
            print(f"  {i+1:2d}. '{ending}': {prob:.1%} ({def_count}/{total}) - {examples}")

def analyze_coverage_by_frequency(filename, n=2):
    """
    Анализирует покрытие определений в зависимости от частоты окончаний
    """
    stats = analyze_definition_endings(filename, n, min_count=1)
    
    # Сортируем по убыванию частоты (total)
    stats_by_freq = sorted(stats, key=lambda x: x[2], reverse=True)
    
    total_definitions = sum(def_count for _, _, _, def_count in stats)
    cumulative_defs = 0
    cumulative_words = 0
    
    print(f"\n{'='*70}")
    print(f"ПОКРЫТИЕ ОПРЕДЕЛЕНИЙ ПО ЧАСТОТЕ ОКОНЧАНИЙ (n={n})")
    print(f"{'='*70}")
    print(f"{'Топ-N':<8} {'Окончаний':<12} {'Покрытие':<12} {'Накоп. покрытие':<16} {'Ср. точность':<12}")
    print("-" * 70)
    
    checkpoints = [1, 3, 5, 10, 20, 30, 50, 100, 200, 300, 400, 500]
    for checkpoint in checkpoints:
        if checkpoint > len(stats_by_freq):
            break
            
        top_endings = stats_by_freq[:checkpoint]
        covered_defs = sum(def_count for _, _, _, def_count in top_endings)
        covered_words = sum(total for _, _, total, _ in top_endings)
        
        coverage = covered_defs / total_definitions
        avg_accuracy = sum(prob * total for _, prob, total, _ in top_endings) / covered_words
        
        print(f"{checkpoint:<8} {checkpoint:<12} {coverage:>11.1%} {coverage:>15.1%} {avg_accuracy:>11.1%}")

In [18]:
filename = "dict.opcorpora.txt"  # Укажите путь к вашему файлу
# Детальный анализ для окончаний длиной 2 буквы
print_detailed_analysis(filename, n=3, top_k=10)




Топ-10 окончаний длиной 3 букв с наибольшей вероятностью быть определением:
Окончание  Вероятность  Определения  Всего    Примеры
----------------------------------------------------------------------
'егл':  100.0%           1         1    БЕГЛ
'ысо':  100.0%           2         2    БЕЛОБРЫСО, ЛЫСО
'тыж':  100.0%           1         1    БЕССТЫЖ
'зён':  100.0%          19        19    ВВЕЗЁН, ВЕЗЁН
'зуч':  100.0%           3         3    ВЕЗУЧ, НЕВЕЗУЧ
'сех':  100.0%           3         3    ВСЕХ, ВСЕХ
'етх':  100.0%           1         1    ВЕТХ
'нюч':  100.0%           2         2    ВОНЮЧ, ЛИНЮЧ
'муч':  100.0%           2         2    ГРЕМУЧ, ДРЕМУЧ
'язо':  100.0%           1         1    ДОЛГОВЯЗО

Общая статистика:
Всего слов проанализировано: 5136405
Всего определений найдено: 2153334
Общая вероятность: 41.9%

АНАЛИЗ ЭФФЕКТИВНОСТИ ПРИ РАЗНЫХ ПОРОГАХ ТОЧНОСТИ
Порог    Окончаний    Покрытие     Точность     Эффективность  
---------------------------------------------------------

In [ ]:
filename = "dict.opcorpora.txt"  # Укажите путь к вашему файлу
    
# Основной анализ
#print_detailed_analysis(filename, n=3, top_k=15)

# Анализ покрытия по частоте
analyze_coverage_by_frequency(filename, n=4)

# Сравнение для разных длин окончаний
print(f"\n{'='*70}")
print("СРАВНЕНИЕ ДЛЯ РАЗНЫХ ДЛИН ОКОНЧАНИЙ")
print(f"{'='*70}")

for n in [2, 3, 4]:
    stats = analyze_definition_endings(filename, n, min_count=1)
    total_definitions = sum(def_count for _, _, _, def_count in stats)
    total_words = sum(total for _, total, _, _ in stats)
    
    # Находим оптимальную эффективность
    best_efficiency = 0
    for threshold in [i/100 for i in range(50, 96, 5)]:
        high_prob_endings = [(ending, prob, total, def_count) 
                           for ending, prob, total, def_count in stats 
                           if prob >= threshold]
        
        if not high_prob_endings:
            continue
            
        total_covered_words = sum(total for _, _, total, _ in high_prob_endings)
        total_covered_defs = sum(def_count for _, _, _, def_count in high_prob_endings)
        
        coverage = total_covered_defs / total_definitions if total_definitions > 0 else 0
        weighted_accuracy = sum(prob * total for _, prob, total, _ in high_prob_endings) / total_covered_words
        efficiency = weighted_accuracy * coverage
        
        if efficiency > best_efficiency:
            best_efficiency = efficiency
    
    print(f"Длина {n}: оптимальная эффективность = {best_efficiency:.1%}")